# LangChain 实战：化妆品销售聊天机器人

In [6]:
import os
os.environ['OPENAI_API_KEY'] = 'XXX'
os.environ['OPENAI_BASE_URL'] = 'XXX'

In [7]:
## 读取化妆品数据
with open("real_cosmetics_sales_data.text", encoding="utf-8") as f:
    real_cosmetics_sales = f.read()

In [20]:
from langchain.text_splitter import CharacterTextSplitter

In [21]:
## 进行文本分割
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [23]:
docs = text_splitter.create_documents([real_cosmetics_sales])

In [24]:
docs[1]

Document(page_content='[客户问题]：这款产品有抗衰老效果吗？\n[销售回答]：是的，这款产品含有多种抗氧化成分，可以帮助减少细纹和皱纹，提升皮肤的弹性和紧致度。')

In [25]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储化妆品销售 问答对（QA-Pair）

In [26]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [14]:
query = "你们的产品是否有天然成分"

In [15]:
answer_list = db.similarity_search(query)

In [16]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：你们的产品是否有天然成分？
[销售回答]：我们的产品中含有大量天然成分，如植物提取物和天然油脂，确保温和无刺激，适合各种肤质。

[客户问题]：这款产品的成分安全吗？
 [销售回答]：我们的产品经过严格的质量控制和安全测试，所有成分均符合国际标准，您可以放心使用。

[客户问题]：你们的产品有没有动物测试？
[销售回答]：我们的产品全部经过严格的安全测试，不进行任何动物实验，您可以放心使用。

[客户问题]：这款产品会引起过敏吗？
 [销售回答]：我们的产品经过敏测试，成分温和，不易引起过敏，适合敏感肌肤使用。



In [17]:
db.save_local("real_cosmetics_sales")

#### 当向量数据库中没有合适答案时，使用大语言模型能力，且提供prompt对检索不到的问题进行回答

In [65]:
from langchain.chains import RetrievalQA
from langchain import LLMChain

# 处理查询和备用逻辑
def custom_retrieval_qa(question, retriever, llm, fallback_prompt):
    # 检索器查找答案
    docs = retriever.get_relevant_documents(question)
    
    # 如果查到对应文档，根据文档生成答案
    if docs:
        qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
        return qa_chain.run(question)
    
    # 如果没查到文档，使用备用提示模板生成答案
    else:
        fallback_chain = LLMChain(prompt=fallback_prompt, llm=llm, output_key="fallback", verbose=True)
        return fallback_chain.invoke({'question': question}).get('fallback', '').strip()

In [66]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)

template = """你是一位化妆品销售。根据传入的问题，对问题做出回答。
问题：{question}
答案：以下是对上述问题的回答："""

fallback_prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

question = "动物实验"

# 示例检索器
retriever = db.as_retriever(search_type="similarity_score_threshold",
                            search_kwargs={"score_threshold": 0.8})


answer = custom_retrieval_qa(question, retriever, llm, fallback_prompt)

print(answer)

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
你是一位化妆品销售。根据传入的问题，对问题做出回答。
问题：动物实验
答案：以下是对上述问题的回答：

> Finished chain.
我们的公司非常重视动物的权益，我们的所有产品都没有进行过动物实验。我们坚决反对任何形式的动物虐待，包括动物实验。我们的产品安全性检测都是通过科学的实验室方法进行的，确保产品的安全性和有效性。
